<a href="https://colab.research.google.com/github/vamsiwillbe/Digantara_solution/blob/main/Sat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

satellite # New Section

In [ ]:
pip install sgp4


# New Section

In [ ]:
pip install sgp4

In [ ]:
from sgp4.api import Satrec, jday
import pandas as pd

In [ ]:
import pandas as pd

In [ ]:
tle_file = "/content/sample_data/30sats.txt"
with open(tle_file, 'r') as tle_data:
    tle_lines = tle_data.read().splitlines()

In [ ]:
time_positions = []
satellites = []

start_year, start_month, start_day, start_hour, start_minute, start_second = 2023, 1, 1, 0, 0, 0
end_year, end_month, end_day, end_hour, end_minute, end_second = 2023, 1, 5, 0, 0, 0

for tle_line in tle_lines:
    # Extract satellite name, line 1, and line 2
    satellite_name = tle_line.strip()
    line1 = tle_lines.pop(0)
    line2 = tle_lines.pop(0)

    satellite = Satrec.twoline2rv(line1, line2)
    satellites.append(satellite)

    current_time = pd.Timestamp(year=start_year, month=start_month, day=start_day,
                                hour=start_hour, minute=start_minute, second=start_second)
    end_time = pd.Timestamp(year=end_year, month=end_month, day=end_day,
                           hour=end_hour, minute=end_minute, second=end_second)

    while current_time <= end_time:
        jd, fr = jday(current_time.year, current_time.month, current_time.day,
                      current_time.hour, current_time.minute, current_time.second)

        e, r, v = satellite.sgp4(jd, fr)

        time_positions.append([current_time, *r, *v])

        current_time += pd.Timedelta(seconds=1)

In [ ]:
column_names = ['Time', 'Px', 'Py', 'Pz', 'Vx', 'Vy', 'Vz']
df = pd.DataFrame(time_positions, columns=column_names)
df.to_csv('satellite_state_vectors.csv', index=False)

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('satellite_state_vectors.csv')

In [ ]:
print(data)


                        Time           Px           Py           Pz        Vx  \
0        2023-01-01 00:00:00 -5012.011414  3701.575225 -4060.546042 -4.643448   
1        2023-01-01 00:00:01 -5016.652219  3695.500787 -4062.125062 -4.638593   
2        2023-01-01 00:00:02 -5021.288168  3689.422772 -4063.700144 -4.633735   
3        2023-01-01 00:00:03 -5025.919259  3683.341186 -4065.271287 -4.628875   
4        2023-01-01 00:00:04 -5030.545488  3677.256036 -4066.838491 -4.624012   
...                      ...          ...          ...          ...       ...   
6912015  2023-01-04 23:59:56  4909.662585  -146.273819 -5006.876596 -1.428209   
6912016  2023-01-04 23:59:57  4908.231541  -139.058958 -5008.463244 -1.433868   
6912017  2023-01-04 23:59:58  4906.794838  -131.843937 -5010.044103 -1.439525   
6912018  2023-01-04 23:59:59  4905.352479  -124.628764 -5011.619170 -1.445181   
6912019  2023-01-05 00:00:00  4903.904465  -117.413447 -5013.188444 -1.450835   

               Vy        Vz

In [ ]:
from google.colab import files

In [ ]:
!pip install pyproj


In [ ]:
import pyproj

def ecef2lla(pos_x, pos_y, pos_z):
    ecef = pyproj.CRS("EPSG:4978")  # ECEF coordinate system
    lla = pyproj.CRS("EPSG:4326")  # WGS84 (Lat, Long, Alt) coordinate system
    transformer = pyproj.Transformer.from_crs(ecef, lla, always_xy=True)

    lla_data = []

    for i in range(len(pos_x)):
        lon, lat, alt = transformer.transform(pos_x[i], pos_y[i], pos_z[i])
        lla_data.append([lon, lat, alt])

    return lla_data

# Assuming 'data' is your DataFrame containing the ECEF position data
ecef_data = data[['Px', 'Py', 'Pz']]
converted_data = ecef2lla(ecef_data['Px'], ecef_data['Py'], ecef_data['Pz'])

# Create a new DataFrame for the [Lat, Long, Alt] data
lla_df = pd.DataFrame(converted_data, columns=['Longitude', 'Latitude', 'Altitude'])

# Concatenate the [Lat, Long, Alt] data with the original data
final_data = pd.concat([data, lla_df], axis=1)

# Print the final data with [Lat, Long, Alt]
print(final_data)


                        Time           Px           Py           Pz        Vx  \
0        2023-01-01 00:00:00 -5012.011414  3701.575225 -4060.546042 -4.643448   
1        2023-01-01 00:00:01 -5016.652219  3695.500787 -4062.125062 -4.638593   
2        2023-01-01 00:00:02 -5021.288168  3689.422772 -4063.700144 -4.633735   
3        2023-01-01 00:00:03 -5025.919259  3683.341186 -4065.271287 -4.628875   
4        2023-01-01 00:00:04 -5030.545488  3677.256036 -4066.838491 -4.624012   
...                      ...          ...          ...          ...       ...   
6912015  2023-01-04 23:59:56  4909.662585  -146.273819 -5006.876596 -1.428209   
6912016  2023-01-04 23:59:57  4908.231541  -139.058958 -5008.463244 -1.433868   
6912017  2023-01-04 23:59:58  4906.794838  -131.843937 -5010.044103 -1.439525   
6912018  2023-01-04 23:59:59  4905.352479  -124.628764 -5011.619170 -1.445181   
6912019  2023-01-05 00:00:00  4903.904465  -117.413447 -5013.188444 -1.450835   

               Vy        Vz

In [ ]:
import pandas as pd


In [ ]:
filtered_data = []

for region in regions:
    latitude1 = region['latitude1']
    longitude1 = region['longitude1']
    latitude2 = region['latitude2']
    longitude2 = region['longitude2']

    # Filter data within the region
    region_data = final_data[
        (final_data['Latitude'] >= min(latitude1, latitude2)) &
        (final_data['Latitude'] <= max(latitude1, latitude2)) &
        (final_data['Longitude'] >= min(longitude1, longitude2)) &
        (final_data['Longitude'] <= max(longitude1, longitude2))
    ]

    # Append filtered data to the result
    filtered_data.append(region_data)

# You now have a list of DataFrames, one for each region


In [ ]:
for i, region_data in enumerate(filtered_data):
    print(f"Region {i+1} Data:")
    print(region_data)


Region 1 Data:
                        Time           Px           Py           Pz        Vx  \
2002     2023-01-01 00:33:22 -4903.195603 -8281.576051 -1231.597129  3.635955   
2003     2023-01-01 00:33:23 -4899.558627 -8285.147602 -1228.441472  3.638095   
2004     2023-01-01 00:33:24 -4895.919512 -8288.715535 -1225.285278  3.640234   
2005     2023-01-01 00:33:25 -4892.278260 -8292.279849 -1222.128546  3.642370   
2006     2023-01-01 00:33:26 -4888.634872 -8295.840545 -1218.971280  3.644504   
...                      ...          ...          ...          ...       ...   
6912015  2023-01-04 23:59:56  4909.662585  -146.273819 -5006.876596 -1.428209   
6912016  2023-01-04 23:59:57  4908.231541  -139.058958 -5008.463244 -1.433868   
6912017  2023-01-04 23:59:58  4906.794838  -131.843937 -5010.044103 -1.439525   
6912018  2023-01-04 23:59:59  4905.352479  -124.628764 -5011.619170 -1.445181   
6912019  2023-01-05 00:00:00  4903.904465  -117.413447 -5013.188444 -1.450835   

            